# Protocol for a strong coin flip

## Problem: 
Let’s pretend for a moment that Alice and Bob are on the telephone and they have decided to flip a coin just for fun. Because they are over the telephone, Alice can’t see Bobs’s coin and vice versa. Additionally, let’s assume that both of them have access to a quantum computer and none of them want to cheat. (for the sake of the problem). This assumption is necessary in order to get a useful result. We’ll see why at the end of the post. For now let’s just concentrate on the process they have to follow:

First let's define some imports.

In [1]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from random import randint
import hashlib

In the next cells Bob and Alice make an exchange of random bits using the bit commitment protocol. If you don’t know the bit exchange protocol you can look it up here.


In the next cell, Alice chooses the bit She wants to send to Bob then She encrypts Her decision and sends it to Bob. The encryption works as follows: She chooses a random number between 0 and 1 if the number is 1 She encrypts the chosen bit, and She leaves the bit without encryption otherwise. We’ll call the random number Alice Basis. Notice that it would be impossible for Bob to know if the bit was encrypted or not.

In [2]:
#Alice cell, Bob can't see what's going in on here
Alice_base = randint(0,1)
qubits = list()
mycircuit = QuantumCircuit(1,1)
if(Alice_base == 0):
    mycircuit.x(0)
else:
    mycircuit.x(0)
    mycircuit.h(0)
qubits.append(mycircuit)

Then, Bob receives the qubit from Alice and after a while She sends Her basis to him. Then Bob proceeds to decipher the original message. This consists in applying an H gate if Alice’s basis says 1. After applying these gates Bob will do a measurement and He’ll be able to recover the original bit.

In [3]:
#Bob cell, Alice can't see what's going in on here
backend = Aer.get_backend('qasm_simulator')
a = 0
qubit = qubits[0]
if(Alice_base == 0):
    qubit.measure(0,0)
else:
    qubit.h(0)
    qubit.measure(0,0)
result = execute(qubit, backend, shots=1, memory=True).result()
a = int(result.get_memory()[0])

Now Bob have to do the same process as Alice. He needs to send Alice an encrypted bit to Alice.

In [4]:
#Bob cell, Alice can't see what's going in on here
Bob_base = randint(0,1)
qubitsB = list()
mycircuitB = QuantumCircuit(1,1)
if(Bob_base == 0):
    mycircuitB.x(0)
else:
    mycircuitB.x(0)
    mycircuitB.h(0)
qubitsB.append(mycircuitB)

Then Alice decrypts Bob’s chosen bit.

In [5]:
#Alice cell, Bob can't see what's going in on here
backend = Aer.get_backend('qasm_simulator')
b = 0
qubit = qubitsB[0]
if(Bob_base == 0):
    qubit.measure(0,0)
else:
    qubit.h(0)
    qubit.measure(0,0)
result = execute(qubit, backend, shots=1, memory=True).result()
b = int(result.get_memory()[0])

Finally in order to obtain their shared coin flip they calculate the XOR between their two bits. They do an XOR because the result of the operations will be the same. And by the end of the protocol they’ll have a random shared bit. Or the equivalent, a coin flip.

In [6]:
#Alice cell, Bob can't see what's going in on here
print("The result of the coin flip was: " + str(b ^ a))

The result of the coin flip was: 0


In [7]:
#Bob cell, Alice can't see what's going in on here
print("The result of the coin flip was: " + str(b ^ a))

The result of the coin flip was: 0


As you can see this protocol is just an extension of the bit commitment protocol. You may even say that is an application of that protocol. But what happens when one or the two of them wants to cheat in the protocol? We’ll see that in a moment.

## Why it doesn’t work when one of them cheats?

Suppose that one of them want to cheat, for example Bob wants to obtain a 0 all the times. One of the places where He could cheat is when He apply the XOR. But since Alice played by the rules, Alice bit will be random and the result of the protocol will be random. So, Alice result will be different of Bob’s bit and then She’ll immediately know that Bob has cheated on the protocol. In other words, Bob can’t force Alice Bit to be wherever He chooses because Alice will immediately notice the change. Another time where Bob could have chosen to cheat was in the bit agreement protocol. In order to cheat He would choose to lie to Alice about their shared key (a, b bits). However, Alice will notice because both of them have the same information.

And what about the case when both of them choose to cheat? In that case, the protocol fails because Alice would obtain a different bit compared to Bob’s bits and vice versa so none of them would obtain a truly random number. But thanks to our initial supposition this case will never happen.

# References:
Blum M.: Coin flipping by telephone a protocol for solving impossible problems. ACM SIGACT News
15(1), 23–27 (1983). doi:10.1145/1008908.1008911.
Broadbent, A., Schaffner, C. Quantum cryptography beyond quantum key distribution. Des. Codes Cryptogr. 78, 351–382 (2016). https://doi.org/10.1007/s10623-015-0157-4